# Spark demo with rotten tomatoes data set from kaggle

In [1]:
% matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
pd.set_option('display.width',500)
pd.set_option('display.max_columns',100)

In [2]:
import findspark
findspark.init()
import pyspark
sc= pyspark.SparkContext()

In [3]:
sc

In [15]:
(sc.parallelize([1,2,3,4,5,6,7,8,9,10]).map(lambda x:x**2).max())
# many functional opearations supported ,advantage over hadoop/mrjob

100

In [16]:
data = sc.textFile('kaggle_rottenTomto.tsv')
data

MapPartitionsRDD[31] at textFile at NativeMethodAccessorImpl.java:-2

Actions returns a value to the driver program, transformation transfrorms from RDD1 to RDD2. count() is an example for action

In [17]:
data.count()

156061

The columns in the tsv are ***PhraseId***, ***SentenceId***, ***Phrase***, ***Sentiment***

In [18]:
#negative_reviews = data.filter(lambda x:x[3]=='0')
#print negative_reviews 
#[u'1000\t36\ttry to avoid\t1'] => all lines like these which have 0 in 4th place get filtered by above filter\
negative_reviews = (data.map(lambda x: x.split('\t')).filter(lambda x:x[3]=='0'))
#same can be achieved by negative_reviews = data.filter(lambda x: x.split('\t')[3]=='0')
negative_reviews

PythonRDD[33] at RDD at PythonRDD.scala:43

as we can see the transformations are not printed. Tranformations are lazy and are only computed when an action is demanded. but spark gives option to cache or save the data set, which is useful for iterative tasks.

actions get printed

In [19]:
print  negative_reviews.first()
print "\n head \n"
negative_reviews.take(5)

[u'102', u'3', u'would have a hard time sitting through this one', u'0']

 head 



[[u'102', u'3', u'would have a hard time sitting through this one', u'0'],
 [u'104', u'3', u'have a hard time sitting through this one', u'0'],
 [u'158',
  u'5',
  u'Aggressive self-glorification and a manipulative whitewash',
  u'0'],
 [u'160', u'5', u'self-glorification and a manipulative whitewash', u'0'],
 [u'202', u'7', u'Trouble Every Day is a plodding mess .', u'0']]

In [20]:
negative_reviews

PythonRDD[33] at RDD at PythonRDD.scala:43

In [21]:
negative_reviews.count()

7072

In [23]:
(negative_reviews.flatMap(lambda x: x[2].split(" "))
 .map(lambda x:(x,1))
 .reduceByKey(lambda x,y : x+y)
 .map(lambda (x,y):(y,x))
 .top(5))

[(3722, u','), (3070, u'the'), (2572, u'a'), (2507, u'and'), (2236, u'of')]

## save the data 

In [25]:
negative_reviews.saveAsTextFile("kaggle_rottenTmts_neg_reviews.txt")

In [27]:
neg_rev = sc.textFile("kaggle_rottenTmts_neg_reviews.txt/")

In [28]:
neg_rev.count()

7072